In [22]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

from langchain.pydantic_v1 import BaseModel, Field

from typing import Optional


#### OUTPUT SCHEMA ####
class LockState(BaseModel):
    color: Optional[str] = Field(..., description="FORMAT TEXT COLORS TO HEX. First color found. If no color is found then return none. Format the color to hex format before returning.")
    lock_type: Optional[str] = Field(..., description="The type of lock to be updated. The VALUES to choose from: ['open', 'closed', 'none', 'half-opened']")

system_prompt = """
You are an expert extraction algorithm.
Only extract relevant information from the text.
If you do not know the value of an attribute asked to extract,
return null for the attribute's value.

Parse the text and find a color and lock type if present.

Turn all colors to hex format.

TAKE YOUR TIME TO THINK
"""

# 1 init model
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 2 create prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)
# 2 create chat History
chat_history = ChatMessageHistory()
chat_model_with_tools = model.with_structured_output(LockState)
chat_chain = prompt | model

chat_chain_with_history = RunnableWithMessageHistory(
    chat_chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [23]:
chat_chain_with_history.invoke({"input": "Turn the lock to black and closed"}, {'configurable': {"session_id": "123"}})

AIMessage(content='{\n    "color": "#000000",\n    "lock_type": "closed"\n}', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 83, 'total_tokens': 101}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-3760d136-2bd7-4d29-8ca3-deee3505829e-0')

In [19]:
chat_chain_with_history.invoke({"input": "Turn the lock green and opened it up."}, {'configurable': {"session_id": "123"}})

AIMessage(content='{\n  "color": "#008000",\n  "state": "opened"\n}', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 116, 'total_tokens': 133}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-ca43e675-14cb-4366-b364-1700078888d1-0')

In [21]:
chat_chain_with_history.invoke({"input": "use the the second = message I sent"}, {'configurable': {"session_id": "123"}})

AIMessage(content='{\n  "color": "#008000",\n  "state": "opened"\n}', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 181, 'total_tokens': 198}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-dcddf4d6-703b-46d8-a10f-0a82ab0ea278-0')